In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
import os
import glob
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import neighbors
from sklearn import datasets, linear_model, metrics

from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_colwidth', None)

np.random.seed(0)

In [3]:
# converts dataframe into numpy array of correct shape
def convert(df: pd.DataFrame, frame_size=10):
    x = np.asarray(df.iloc[:, 1]).reshape(-1, frame_size)
    return x

In [4]:
######################## Load Datasets ####

data_dir = Path("../../Datasets/Smartphone/50cm")
df1 = pd.read_csv(data_dir / "VentOne_0degrees_50cm.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df1 = df1[df1['frequency'] != "clocktime"]
df1 = df1.dropna()
df1 = convert(df1)

df2 = pd.read_csv(data_dir / "VentOne_180degrees_50cm.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df2 = df2[df2['frequency'] != "clocktime"]
df2 = df2.dropna()
df2 = convert(df2)

df3 = pd.read_csv(data_dir / "VentOne_90degrees_50cm.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df3 = df3[df3['frequency'] != "clocktime"]
df3 = df3.dropna()
df3 = convert(df3)


data_dir = Path("../../Datasets/Smartphone/1m")
df4 = pd.read_csv(data_dir / "VentOne_0degrees_1meter.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df4 = df4[df4['frequency'] != "clocktime"]
df4 = df4.dropna()
df4 = convert(df4)

df5 = pd.read_csv(data_dir / "VentOne_180degrees_1meter.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df5 = df5[df5['frequency'] != "clocktime"]
df5 = df5.dropna()
df5 = convert(df5)

df6 = pd.read_csv(data_dir / "VentOne_90degrees_1meter.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df6 = df6[df6['frequency'] != "clocktime"]
df6 = df6.dropna()
df6 = convert(df6)


data_dir = Path("../../Datasets/Smartphone/1.5m")
df7 = pd.read_csv(data_dir / "VentOne_0degrees_1-5meter.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df7 = df7[df7['frequency'] != "clocktime"]
df7 = df7.dropna()
df7 = convert(df7)

df8 = pd.read_csv(data_dir / "VentOne_90degrees_1-5meter.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df8 = df8[df8['frequency'] != "clocktime"]
df8 = df8.dropna()
df8 = convert(df8)

df9 = pd.read_csv(data_dir / "VentOne_180degrees_1-5meter.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df9 = df9[df9['frequency'] != "clocktime"]
df9 = df9.dropna()
df9 = convert(df9)


data_dir = Path("../../Datasets/Smartphone/SideVent")
df10 = pd.read_csv(data_dir / "SideVent_0degrees_0cm.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df10 = df10[df10['frequency'] != "clocktime"]
df10 = df10.dropna()
df10 = convert(df10)


data_dir = Path("../../Datasets/Smartphone/2.25m")
df11 = pd.read_csv(data_dir / "VentOne_0degrees_2-25meters.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df11 = df11[df11['frequency'] != "clocktime"]
df11 = df11.dropna()
df11 = convert(df11)

df12 = pd.read_csv(data_dir / "VentTwo_90degrees_2-25meters.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df12 = df12[df12['frequency'] != "clocktime"]
df12 = df12.dropna()
df12 = convert(df12)

df13 = pd.read_csv(data_dir / "VentOne_180degrees_2-25meters.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df13 = df13[df13['frequency'] != "clocktime"]
df13 = df13.dropna()
df13 = convert(df13)

df14 = pd.read_csv(data_dir / "VentOne_0degrees_2-25meters.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df14 = df14[df14['frequency'] != "clocktime"]
df14 = df14.dropna()
df14 = convert(df14)


data_dir = Path("../../Datasets/Smartphone/NoAirflowData")
df15 = pd.read_csv(data_dir / "NoAir_Dataset1.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df15 = df15[df15['frequency'] != "clocktime"]
df15 = df15.dropna()
df15 = convert(df15)

df16 = pd.read_csv(data_dir / "NoAir_Dataset2.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df16 = df16[df16['frequency'] != "clocktime"]
df16 = df16.dropna()
df16 = convert(df16)

In [5]:
def indices(df):
    return df[2000:7000]

ones = np.ones(5000).reshape(-1, 1)
zeroes = np.zeros(20000).reshape(-1, 1)

# CREATE LABELED DATA W/ AIRFLOW

sf1 = np.append(indices(df1), ones, 1)
sf4 = np.append(indices(df4), ones, 1)
sf5 = np.append(indices(df5), ones, 1)
sf6 = np.append(indices(df6), ones, 1)
sf7 = np.append(indices(df7), ones, 1)
sf8 = np.append(indices(df8), ones, 1)
sf11 = np.append(indices(df11), ones, 1)
sf14 = np.append(indices(df14), ones, 1)

# CREATE LABELED DATA W/OUT AIRFLOW
sf15 = np.append(df15[:20000], zeroes, 1)
sf16 = np.append(df16[:20000], zeroes, 1)

df = np.concatenate([sf1, sf4, sf5, sf6, sf7, sf8, sf11, sf14, sf15, sf16], axis=0)

np.random.shuffle(df)
print(df.shape)

(80000, 11)


In [6]:
######################## Load Model from .joblib file ####
from joblib import dump,load
clf = load('flowSenseClassifier.joblib')

/Users/adam/miniforge3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator DummyClassifier from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/adam/miniforge3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/adam/miniforge3/lib/python3.8/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator GradientBoostingClassifier from version 0.24.2 when using version 1.0.1. This migh

In [18]:
from pathlib import Path

scores_2_0 = []
scores_2_90 = []
scores_2_180 = []

############# Looking at classifications from 50cm
data_dir = Path("../../Datasets/Smartphone/50cm")
df1 = pd.read_csv(data_dir / "VentOne_0degrees_50cm.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df1 = df1[df1['frequency'] != "clocktime"]
df1 = df1.dropna()

df2 = pd.read_csv(data_dir / "VentOne_180degrees_50cm.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df2 = df2[df2['frequency'] != "clocktime"]
df2 = df2.dropna()

df3 = pd.read_csv(data_dir / "VentOne_90degrees_50cm.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df3 = df3[df3['frequency'] != "clocktime"]
df3 = df3.dropna()

df = convert(df1)
for indices in [range(4000,5000), range(5000,6000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_2_0.append(clf.score(X, y))

df = convert(df2)
for indices in [range(4000,5000), range(5000,6000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_2_90.append(clf.score(X, y))

df = convert(df3)
for indices in [range(4000,5000), range(5000,6000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_2_180.append(clf.score(X, y))

print("50cm, 0 degrees")
print(scores_2_0)

print("50cm, 90 degrees")
print(scores_2_90)

print("50cm, 180 degrees")
print(scores_2_180)

50cm, 0 degrees
[0.991, 0.983, 0.998, 0.978]
50cm, 90 degrees
[0.97, 0.965, 0.964, 0.962]
50cm, 180 degrees
[0.967, 0.959, 0.96, 0.962]


In [19]:
scores_2_0 = []
scores_2_90 = []
scores_2_180 = []

############# Looking at classifications from 1m
data_dir = Path("../../Datasets/Smartphone/1m")
df1 = pd.read_csv(data_dir / "VentOne_0degrees_1meter.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df1 = df1[df1['frequency'] != "clocktime"]
df1 = df1.dropna()

df2 = pd.read_csv(data_dir / "VentOne_90degrees_1meter.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df2 = df2[df2['frequency'] != "clocktime"]
df2 = df2.dropna()

df3 = pd.read_csv(data_dir / "VentOne_180degrees_1meter.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df3 = df3[df3['frequency'] != "clocktime"]
df3 = df3.dropna()

df = convert(df1)
for indices in [range(4000,5000), range(5000,6000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_2_0.append(clf.score(X, y))

df = convert(df2)
for indices in [range(4000,5000), range(5000,6000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_2_90.append(clf.score(X, y))

df = convert(df3)
for indices in [range(4000,5000), range(5000,6000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_2_180.append(clf.score(X, y))

print("1m, 0 degrees")
print(scores_2_0)

print("1m, 90 degrees")
print(scores_2_90)

print("1m, 180 degrees")
print(scores_2_180)

1m, 0 degrees
[0.948, 0.956, 0.971, 0.935]
1m, 90 degrees
[0.962, 0.961, 0.944, 0.919]
1m, 180 degrees
[0.965, 0.971, 0.974, 0.91]


In [17]:
scores_2_0 = []
scores_2_90 = []
scores_2_180 = []

############# Looking at classifications from 1.5m
data_dir = Path("../../Datasets/Smartphone/1.5m")
df1 = pd.read_csv(data_dir / "VentOne_0degrees_1-5meter.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df1 = df1[df1['frequency'] != "clocktime"]
df1 = df1.dropna()

df2 = pd.read_csv(data_dir / "VentOne_90degrees_1-5meter.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df2 = df2[df2['frequency'] != "clocktime"]
df2 = df2.dropna()

df3 = pd.read_csv(data_dir / "VentOne_180degrees_1-5meter.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df3 = df3[df3['frequency'] != "clocktime"]
df3 = df3.dropna()

df = convert(df1)
for indices in [range(4000,5000), range(5000,6000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_2_0.append(clf.score(X, y))

df = convert(df2)
for indices in [range(4000,5000), range(5000,6000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_2_90.append(clf.score(X, y))

df = convert(df3)
for indices in [range(4000,5000), range(5000,6000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_2_180.append(clf.score(X, y))

print("1.5m, 0 degrees")
print(scores_2_0)

print("1.5m, 90 degrees")
print(scores_2_90)

print("1.5m, 180 degrees")
print(scores_2_180)

1.5m, 0 degrees
[0.96, 0.947, 0.956, 0.917]
1.5m, 90 degrees
[0.886, 0.886, 0.889, 0.806]
1.5m, 180 degrees
[0.822, 0.836, 0.794, 0.836]


In [15]:
scores_2_0 = []
scores_2_90 = []
scores_2_180 = []

############# Looking at classifications from 2.25m
data_dir = Path("../../Datasets/Smartphone/2.25m")
df1 = pd.read_csv(data_dir / "VentOne_0degrees_2-25meters.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df1 = df1[df1['frequency'] != "clocktime"]
df1 = df1.dropna()

df2 = pd.read_csv(data_dir / "VentOne_90degrees_2-25meters.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df2 = df2[df2['frequency'] != "clocktime"]
df2 = df2.dropna()

df3 = pd.read_csv(data_dir / "VentOne_180degrees_2-25meters.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df3 = df3[df3['frequency'] != "clocktime"]
df3 = df3.dropna()

df = convert(df1)
for indices in [range(2000,3000), range(3000,4000), range(4000,5000), range(5000,6000), range(6000,7000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_2_0.append(clf.score(X, y))

df = convert(df2)
for indices in [range(1000,2000), range(2000,3000), range(3000,4000), range(4000,5000), range(5000,6000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_2_90.append(clf.score(X, y))

df = convert(df3)
for indices in [range(2000,3000), range(3000,4000), range(4000,5000), range(5000,6000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_2_180.append(clf.score(X, y))

print("2.25m, 0 degrees")
print(scores_2_0)

print("2.25m, 90 degrees")
print(scores_2_90)

print("2.25m, 180 degrees")
print(scores_2_180)

2.25m, 0 degrees
[0.916, 0.904, 0.918, 0.928, 0.922]
2.25m, 90 degrees
[0.916, 0.962, 0.95, 0.953, 0.957, 0.953, 0.938]
2.25m, 180 degrees
[0.774, 0.783, 0.804, 0.872, 0.901]


In [8]:
from pathlib import Path

scores_3_0 = []
scores_3_90 = []
scores_3_180 = []

######################## LOOKING AT ACCURACY ON 3 METER REAL WORLD DATA ####

data_dir = Path("../../Datasets/Smartphone/3m")
df1 = pd.read_csv(data_dir / "VentOne_90degrees_3meters.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df1 = df1[df1['frequency'] != "clocktime"]
df1 = df1[df1['frequency'] != "silence"]
df1 = df1.dropna()

df2 = pd.read_csv(data_dir / "VentOne_180degrees_3meters.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df2 = df2[df2['frequency'] != "clocktime"]
df2 = df2[df2['frequency'] != "silence"]
df2 = df2.dropna()

df3 = pd.read_csv(data_dir / "VentOne_0degrees_3meters.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df3 = df3[df3['frequency'] != "clocktime"]
df3 = df3[df3['frequency'] != "silence"]
df3 = df3.dropna()

df = convert(df1)
for indices in [range(14000,15000), range(2000,3000), range(1000,2000), range(10000,11000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_3_90.append(clf.score(X, y))
    
df = convert(df2)
for indices in [range(4000,5000), range(5000,6000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_3_180.append(clf.score(X, y))
    
df = convert(df3)
for indices in [range(14000,15000), range(15000,16000), range(16000,17000), range(17000,18000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_3_0.append(clf.score(X, y))

print("3m, 0 degrees")
print(scores_3_0)

print("3m, 90 degrees")
print(scores_3_90)

print("3m, 180 degrees")
print(scores_3_180)

3m, 0 degrees
[0.821, 0.801, 0.795, 0.878]
3m, 90 degrees
[0.97, 0.976, 0.969, 0.984]
3m, 180 degrees
[0.509, 0.498, 0.524, 0.547]


In [20]:
from pathlib import Path

scores_4_0 = []
scores_4_90 = []
scores_4_180 = []

######################## LOOKING AT ACCURACY ON 4 METER REAL WORLD DATA ####

data_dir = Path("../../Datasets/Smartphone/4m")
df1 = pd.read_csv(data_dir / "VentOne_0degrees_4meters.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df1 = df1[df1['frequency'] != "clocktime"]
df1 = df1[df1['frequency'] != "silence"]
df1 = df1.dropna()

df2 = pd.read_csv(data_dir / "VentOne_180degrees_4meters.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df2 = df2[df2['frequency'] != "clocktime"]
df2 = df2[df2['frequency'] != "silence"]
df2 = df2.dropna()

df3 = pd.read_csv(data_dir / "VentOne_90degrees_4meters.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df3 = df3[df3['frequency'] != "clocktime"]
df3 = df3[df3['frequency'] != "silence"]
df3 = df3.dropna()

df = convert(df1)
for indices in [range(4000,5000), range(5000,6000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_4_90.append(clf.score(X, y))
    
df = convert(df2)
for indices in [range(4000,5000), range(5000,6000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_4_180.append(clf.score(X, y))
    
df = convert(df3)
for indices in [range(14000,15000), range(15000,16000), range(6000,7000), range(8000,9000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_4_0.append(clf.score(X, y))

print("4m, 0 degrees")
print(scores_4_0)

print("4m, 90 degrees")
print(scores_4_90)

print("4m, 180 degrees")
print(scores_4_180)

4m, 0 degrees
[0.978, 0.974, 0.973, 0.961]
4m, 90 degrees
[0.967, 0.978, 0.969, 0.983]
4m, 180 degrees
[0.975, 0.983, 0.979, 0.971]


In [22]:
scores_P1_5u = []
scores_P1_5o = []
scores_P1_2u = []
scores_P1_2o = []
scores_P1_2h = []

################# LOOKING AT ACCURACY ON REAL WORLD DATA (with and without ambient noise) FROM PHONE 1 ####

data_dir = Path("../../Datasets/Smartphone/RealWorld")
df1 = pd.read_csv(data_dir / "PhoneOne_50cm_unoccupied.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df1 = df1[df1['frequency'] != "clocktime"]
df1 = df1[df1['frequency'] != "silence"]
df1 = df1.dropna()

df2 = pd.read_csv(data_dir / "PhoneOne_50cm_OCCUPIED.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df2 = df2[df2['frequency'] != "clocktime"]
df2 = df2[df2['frequency'] != "silence"]
df2 = df2.dropna()

df3 = pd.read_csv(data_dir / "PhoneOne_2-25m_unoccupied.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df3 = df3[df3['frequency'] != "clocktime"]
df3 = df3[df3['frequency'] != "silence"]
df3 = df3.dropna()

df4 = pd.read_csv(data_dir / "PhoneOne_2-25m_OCCUPIED.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df4 = df4[df4['frequency'] != "clocktime"]
df4 = df4[df4['frequency'] != "silence"]
df4 = df4.dropna()

df5 = pd.read_csv(data_dir / "PhoneOne_InHand_OCCUPIED.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df5 = df5[df5['frequency'] != "clocktime"]
df5 = df5[df5['frequency'] != "silence"]
df5 = df5.dropna()

df = convert(df1)
for indices in [range(1000), range(9000,10000), range(10000,11000), range(11000,12000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_P1_5u.append(clf.score(X, y))

df = convert(df2)
for indices in [range(3000,4000), range(4000,5000), range(5000,6000), range(6000,7000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_P1_5o.append(clf.score(X, y))

df = convert(df3)
for indices in [range(1000), range(1000,2000), range(2000,3000), range(3000,4000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_P1_2u.append(clf.score(X, y))

df = convert(df4)
for indices in [range(1000), range(1000,2000), range(2000,3000), range(3000,4000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_P1_2o.append(clf.score(X, y))

df = convert(df5)
for indices in [range(1000), range(1000,2000), range(2000,3000), range(3000,4000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_P1_2h.append(clf.score(X, y))

print("Phone1 0.5m, UNOCCUPIED SPACE")
print(sum(scores_P1_5u) / len(scores_P1_5u))
print("")
print("Phone1 0.5m, OCCUPIED SPACE")
print(sum(scores_P1_5o) / len(scores_P1_5o))
print("")
print("Phone1 2.25m, UNOCCUPIED SPACE")
print(sum(scores_P1_2u) / len(scores_P1_2u))
print("")
print("Phone1 2.25m, OCCUPIED SPACE")
print(sum(scores_P1_2o) / len(scores_P1_2o))
print("")
print("Phone1 2.25m, IN HAND")
print(sum(scores_P1_2h) / len(scores_P1_2h))

Phone1 0.5m, UNOCCUPIED SPACE
0.8635

Phone1 0.5m, OCCUPIED SPACE
0.8602500000000001

Phone1 2.25m, UNOCCUPIED SPACE
0.9460000000000001

Phone1 2.25m, OCCUPIED SPACE
0.95025

Phone1 2.25m, IN HAND
0.78125


In [24]:
scores_P2_2u = []
scores_P3_2u = []
scores_P4_2u = []
scores_P4_2a = []

################# LOOKING AT ACCURACY ON REAL WORLD DATA (with and without ambient noise) FROM OTHER PHONES ####

data_dir = Path("../../Datasets/Smartphone/RealWorld")
df1 = pd.read_csv(data_dir / "PhoneTwo_2-25m_unoccupied.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df1 = df1[df1['frequency'] != "clocktime"]
df1 = df1[df1['frequency'] != "silence"]
df1 = df1.dropna()

df2 = pd.read_csv(data_dir / "PhoneThree_2-25m_unoccupied.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df2 = df2[df2['frequency'] != "clocktime"]
df2 = df2[df2['frequency'] != "silence"]
df2 = df2.dropna()

df3 = pd.read_csv(data_dir / "PhoneFour_2-25m_unoccupied.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df3 = df3[df3['frequency'] != "clocktime"]
df3 = df3[df3['frequency'] != "silence"]
df3 = df3.dropna()

df4 = pd.read_csv(data_dir / "PhoneFour_2-25m_airflow.csv", low_memory=False, names= ['frequency','amplitude'] , index_col = False)
df4 = df4[df4['frequency'] != "clocktime"]
df4 = df4[df4['frequency'] != "silence"]
df4 = df4.dropna()

df = convert(df1)
for indices in [range(1000), range(1000,2000), range(2000,3000), range(3000,4000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_P2_2u.append(clf.score(X, y))

df = convert(df2)
for indices in [range(1000), range(1000,2000), range(2000,3000), range(3000,4000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_P3_2u.append(clf.score(X, y))

df = convert(df3)
for indices in [range(1000), range(1000,2000), range(2000,3000), range(3000,4000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_P4_2u.append(clf.score(X, y))

df = convert(df4)
for indices in [range(1000), range(1000,2000), range(2000,3000), range(3000,4000)]:
    X = df[:,:-1]
    X = X[indices, :6]
    y = np.ones(1000)
    scores_P4_2a.append(clf.score(X, y))

print("Phone2 2.25m, UNOCCUPIED SPACE")
print(sum(scores_P1_5u) / len(scores_P1_5u))
print("")
print("Phone3 2.25m, UNOCCUPIED SPACE")
print(sum(scores_P1_5o) / len(scores_P1_5o))
print("")
print("Phone4 2.25m, UNOCCUPIED SPACE")
print(sum(scores_P1_2u) / len(scores_P1_2u))
print("")
print("Phone4 2.25m, UNOCCUPIED SPACE w/ more airflow")
print(sum(scores_P1_2o) / len(scores_P1_2o))
print("")

Phone2 2.25m, UNOCCUPIED SPACE
0.8635

Phone3 2.25m, UNOCCUPIED SPACE
0.8602500000000001

Phone4 2.25m, UNOCCUPIED SPACE
0.9460000000000001

Phone4 2.25m, UNOCCUPIED SPACE w/ 
0.95025

